# Registration Experimentation
This Jupyter Notebook evaluates various registration techniques in the development of an animal study pipeline. Babak Ardekani's ART toolset will be employed here with various parameters.

## Import Modules
This section imports various Python modules to process and analyze data for evaluation. The modules being used are responsible for the following functions:
1. subprocess: execute system commands
2. os: manipulate OS-related things
4. numpy: manage and manipulate nifti images as arrays
3. nibabel: read nifti files
4. matplotlib: visualize nifti images

In [3]:
import subprocess
import os
import os.path as op
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

## Docker Configuration
Please configure this section to match your Docker image for ARTDock.

In [4]:
docker_tag = 'artdock:latest'
docker_mount = '/Users/sid/Downloads/Mouse/data'
docker_mountpoint = '/data'

## Path to Files
Define path to various files. There are two types of files to consider:
1. Target: template to register images to
2. Moving: image that will be registered to template

Please enter these paths relative to `docker_mount` variable.

In [5]:
# Target image
trgPath = '/Users/sid/Downloads/Mouse/data/same mice_Time1,2/w1t1m8/ACQ_BRUKER_PVMTurboRARE-T2X4P1.nii'

# Moving image
movPath = '/Users/sid/Downloads/Mouse/data/same mice_Time1,2/w1t2m8/ACQ_BRUKER_PVMTurboRARE-T2X4P1.nii'

# Output Path
outPath = '/Users/sid/Downloads/Mouse/data/out'

### Build Docker Paths
This section uses paths defined for Docker and files to construct mountpounts for inputs and outputs such that they appear to be mounted within a Docker container.

In [33]:
# Separate docker mount path
dockerMountList = [i for i in docker_mount.split(op.sep) if i]

# Separate path to target
trgPathList = [i for i in trgPath.split(op.sep) if i]
trgPathList = [x for x in trgPathList if x not in dockerMountList]
trgPath = op.sep.join(trgPathList)
trgPath = op.join(docker_mountpoint, trgPath)

# Seperate path to moving
movPathList = [i for i in movPath.split(op.sep) if i]
movPathList = [x for x in movPathList if x not in dockerMountList]
movPath = op.sep.join(movPathList)
movPath = op.join(docker_mountpoint, movPath)

# Separate path to out
outPathList = [i for i in outPath.split(op.sep) if i]
outPathList = [x for x in outPathList if x not in dockerMountList]
outPath = op.sep.join(outPathList)
outPath = op.join(docker_mountpoint, outPath)

print(trgPath)
print(movPath)
print(outPath)

/data/same mice_Time1,2/w1t1m8/ACQ_BRUKER_PVMTurboRARE-T2X4P1.nii
/data/same mice_Time1,2/w1t2m8/ACQ_BRUKER_PVMTurboRARE-T2X4P1.nii
/data/out


## Execute Registration
This section calls the ARTDock container and runs ART tools to register the moving image to target template.

In [51]:
# Construct docker command
reg_arg = ['docker', 'run', '-t', '--rm']
# Manage volume mounting
reg_arg.extend(['-v', docker_mount + ':' + docker_mountpoint])
# Execute docker_tag
reg_arg.append(docker_tag)
# Build processing command
# reg_arg.extend(['3dwarper'])
reg_arg.extend(['-v'])
reg_arg.extend(['-iter', '10'])
reg_arg.extend(['-A'])
reg_arg.extend(['-o', op.join(outPath, 'output.nii')])
reg_arg.extend(['-sub', movPath])
reg_arg.extend(['-trg', trgPath])
print('Executing command: ' + ' '.join(reg_arg))
completion = subprocess.run(reg_arg)
if completion.returncode != 0:
    raise Exception('Failed to run docker image {}'.format(docker_tag))

Executing command: docker run -t --rm -v /Users/sid/Downloads/Mouse/data:/data artdock:latest -v -iter 10 -A -o /data/out/output.nii -sub /data/same mice_Time1,2/w1t2m8/ACQ_BRUKER_PVMTurboRARE-T2X4P1.nii -trg /data/same mice_Time1,2/w1t1m8/ACQ_BRUKER_PVMTurboRARE-T2X4P1.nii
